### **1\. Error 객체 구조와 스택 트레이스 (Stack Trace)**  
JavaScript에서 오류가 발생하면 엔진은 단순히 프로그램 실행을 중단하는 것이 아니라, 해당 오류 상황을 상세하게 기술한 Error 객체를 생성합니다.  

**🔷 Error 객체의 핵심 속성**  
**✔️ 예시 코드**

In [ ]:
function thirdFunc() {
    // 정의되지 않은 변수를 참조하여 ReferenceError를 발생시킵니다.
    console.log(nonExistentVariable);
}

function secondFunc() {
    // 세 번째 함수를 호출합니다.
    thirdFunc();
}

function firstFunc() {
    try {
        // 비정상 동작이 발생할 가능성이 있는 함수 호출을 감싸고 있습니다.
        secondFunc();
    } catch (error) {
        console.error("--- 에러 정보 ---");

        // Error 객체의 핵심 속성을 출력합니다.
        // ReferenceError
        console.error("에러 이름 (name):", error.name);
        // nonExistentVariable is not defined
        console.error("에러 메시지 (message):", error.message);
        console.error("\n--- 스택 트레이스 (stack) ---");

        // 에러가 발생한 지점까지의 호출 경로를 보여줍니다.
        console.error(error.stack);
        /*
            출력된 스택 트레이스를 통해
            firstFunc → secondFunc → thirdFunc 순으로 함수가 호출되었고,
            최종적으로 thirdFunc에서 오류가 발생했음을 확인할 수 있습니다.
        */
    }
}

// 전체 호출 흐름을 시작합니다.
firstFunc();

--- 에러 정보 ---
에러 이름 (name): ReferenceError
에러 메시지 (message): nonExistentVariable is not defined

--- 스택 트레이스 (stack) ---
ReferenceError: nonExistentVariable is not defined
    at thirdFunc (<anonymous>:3:15)
    at secondFunc (<anonymous>:7:3)
    at firstFunc (<anonymous>:12:5)
    at <anonymous>:30:1


### **2\. Error.cause (ES2022)**  
ES2022에서 도입된 Error.cause 속성은 이러한 시나리오에서 새로운 오류를 생성할 때, 그 원인이 된 최초의 오류를 체계적으로 연결(Chaining) 할 수 있도록 지원합니다.  

**✔️ Error.cause를 이용한 에러 연결**  

In [ ]:
function callExternalAPI() {
    // 외부 API 호출 시 발생한 에러를 시뮬레이션합니다.
    throw new Error("외부 서버 응답 오류 (HTTP 500)", { cause: 'Connection Timeout' });
}

function handleUserData() {
    try {
        callExternalAPI();
    } catch (externalError) {
        // (1) 외부 에러를 잡습니다.
        console.error("1차 에러: 외부 API 호출 실패");

        // (2) 더 상위 계층으로 던지기 위해 Custom Error로 포장합니다.
        // 이때, 'cause' 속성에 최초의 에러 (externalError)를 담아 정보를 보존합니다.
        throw new Error("사용자 데이터 처리 중 심각한 오류 발생", { cause: externalError });
    }
}

try {
    handleUserData();
} catch (finalError) {
    console.error("\n최종 에러 (사용자에게 표시):", finalError.message);

    // (3) 'cause' 속성을 확인하여 에러의 실제 원인을 추적합니다.
    if (finalError.cause) {
        console.error("--- 에러의 실제 원인 추적 ---");
        console.error("최초 원인 객체:", finalError.cause);
        console.error("최초 원인 메시지:", finalError.cause.message || finalError.cause);
    }
}

### **3\. throw & try/catch 실무 규칙**  
에러 처리는 단순히 에러를 잡는 것을 넘어, 코드의 안전성과 복구 전략을 결정하는 핵심 요소입니다.  

**🔷 1. throw의 역할 - 오류 전파(Propagation)**  
함수가 정상적인 작업을 더 이상 수행할 수 없을 때 에러를 발생시켜 호출자에게 책임을 넘깁니다.  
특히, 잘못된 인수가 전달되거나 데이터베이스 연결 실패 등 '복구 불가능한' 상황에서 사용합니다.  

**🔷 2. try/catch의 역할 - 오류 복구 및 대응**  
 try/catch는 오류 발생 시 복구가 가능한지 여부에 따라 두 가지 방향으로 사용됩니다.  
▸ 복구 가능한 오류 처리  
오류가 발생했지만 대체 로직을 수행하거나 사용자에게 더 친절한 메시지를 전달할 수 있는 경우입니다.  
▸ 로깅 후 재전파(rethrow)  
함수 내부에서 복구할 수 없는 오류라면 해당 함수는 단지 에러를 기록한 후, 오류를 다시 던져 상위 계층에서 적절히 처리하도록 해야 합니다.

**🔷 3. finally의 역할 - 정리(Cleanup) 작업**  
finally 블록은 오류 발생 여부와 관계없이 항상 실행되는 영역으로, 아래와 같은 정리 작업을 수행할 때 필수적입니다.

**✔️ try/catch/finally 실무 패턴**

In [ ]:
function processFile(filePath) {
    let fileHandle; // 파일 리소스를 추적하기 위한 핸들
    try {
        // (1) 파일 경로 유효성 검사
        if (!filePath) {
            // 복구가 불가능한 상황이므로 호출자에게 오류를 전달합니다.
            throw new TypeError("파일 경로를 반드시 지정해야 합니다.");
        }
        // (2) 파일 열기 (예제에서는 가상의 핸들로 대체)
        fileHandle = "File_ID_123";
        console.log("파일을 성공적으로 열었습니다.");
        // (3) 파일 처리 로직이 들어가는 영역
        // ...

    } catch (error) {
        // (4) 에러가 발생했을 때의 처리 로직
        if (error instanceof TypeError) {
            // 예상 가능한 입력 오류 → 로깅 후 복구 처리 가능
            console.error(`[로깅] 입력값 오류: ${error.message}`);
            console.log("기본값 또는 대체 경로로 처리할 수 있습니다.");
            return null; // 호출자에게 작업 실패를 명확히 전달
        } else {
            // 예상치 못한 예외 → 로깅 후 재전파(Fail Fast 전략)
            console.error(`[심각한 오류] 알 수 없는 오류 발생: ${error.message}`);
            throw error; // 호출자에게 다시 전달
        }
    } finally {
        // (5) 에러 발생 여부와 관계없이 반드시 실행되는 정리 작업
        if (fileHandle) {
            console.log("파일 리소스를 정상적으로 닫았습니다. (정리 작업 완료)");
            // fileHandle.close(); // 실제 파일 핸들 해제 코드
        }
    }
}

// 예시 1: 성공 케이스
console.log("--- 성공 케이스 ---");
processFile("/data/file.txt");

// 예시 2: 복구 가능한 오류(TypeError) 발생
console.log("\n--- 복구 가능한 오류 케이스 ---");
processFile(null);

--- 성공 케이스 ---
파일을 성공적으로 열었습니다.
파일 리소스를 정상적으로 닫았습니다. (정리 작업 완료)

--- 복구 가능한 오류 케이스 ---


[로깅] 입력값 오류: 파일 경로를 반드시 지정해야 합니다.


기본값 또는 대체 경로로 처리할 수 있습니다.


null

### **4\. 비동기 에러 처리 (Promise/async/await, 이벤트 error)**  
JavaScript의 비동기 환경(Promise, async/await, EventEmitter 등)에서는 동기 코드의 try/catch만으로는 모든 오류를 처리할 수 없습니다.

**🔷 Promise: .catch()에 의한 오류 처리**  
Promise 체인에서는 오류가 발생하면 가장 가까운 .catch() 블록으로 전달됩니다.  
▸ then 내부에서 오류가 발생해도 .catch()가 처리합니다.  
▸ .catch()가 존재하지 않으면 해당 Promise는 “처리되지 않은 거부(unhandled rejection)” 상태가 됩니다.  
▸ 브라우저와 Node.js 모두 “전역 unhandledrejection 이벤트”를 발생시키며, 이는 실무 로깅에서 반드시 처리해야 하는 중요한 이벤트입니다.



In [3]:
// Promise 체인에서 발생한 오류는 가장 가까운 .catch()로 전달됩니다.
function getData() {
    return new Promise((resolve, reject) => {
        // 의도적으로 오류를 발생시킵니다.
        reject(new Error("데이터 가져오기 실패"));
    });
}

getData()
    .then((data) => {
        console.log("데이터:", data);
    })
    .catch((error) => {
        // reject에서 전달된 오류를 여기서 처리합니다.
        console.error("[Promise 오류 처리]", error.message);
    });

[Promise 오류 처리] 데이터 가져오기 실패


Promise { undefined }

**🔷 async/await: 비동기 코드의 동기적 오류 처리**  
async/await 구문은 Promise를 기반으로 동작하며, await 표현식에서 발생한 오류는 동기 코드와 동일하게 try/catch로 처리할 수 있습니다.


In [ ]:
// async/await 환경에서는 발생한 오류를 try/catch로 처리할 수 있습니다.
async function fetchUser() {
    try {
        // await에서 오류가 발생하면 catch 블록으로 이동합니다.
        const response = await fetch("https://invalid-url.example.com");
        return await response.json();
    } catch (error) {
        console.error("[async/await 오류 처리]", error.message);
        return null; // 프로그램 중단을 방지하기 위해 기본값 반환
    }
}

fetchUser();

**🔷 전역 비동기 오류 처리(unhandledrejection, error)**  
어떠한 이유로든 Promise 오류가 .catch()로 처리되지 않거나 코드 전반에서 예외가 누락될 경우, 시스템은 다음과 같은 전역 이벤트를 발생시킵니다.

브라우저:  
▸ window.addEventListener('unhandledrejection')  
▸ window.addEventListener('error')

Node.js:  
▸ process.on('unhandledRejection')  
▸ process.on('uncaughtException')  

**✔️ 처리되지 않은 Promise 거부(unhandledRejection)**

In [ ]:
// (1) 전역에서 처리되지 않은 Promise.reject() 감지
process.on("unhandledRejection", (reason, promise) => {
    console.error("[Node.js 전역 unhandledRejection 감지]");
    console.error("거부된 Promise:", promise);
    console.error("이유(reason):", reason);

    // 실무에서는 여기서 로깅 및 모니터링 시스템(Sentry, Datadog 등)으로 전송합니다.
    // 예: sendErrorToMonitoringService(reason);

    // 심각한 문제이므로 프로세스를 종료할지 여부를 판단합니다.
    // process.exit(1); // 필요 시 활성화
});

// 의도적으로 .catch()를 생략하여 unhandledRejection 발생
Promise.reject(new Error("전역에서 잡힌 Promise 거부"));

**✔️ 처리되지 않은 일반 예외(uncaughtException)**

In [6]:
// (2) try/catch로 잡히지 않은 오류 전역 감지
process.on("uncaughtException", (error) => {
    console.error("[Node.js 전역 uncaughtException 감지]");
    console.error("오류 메시지:", error.message);
    console.error("스택:", error.stack);

    // 프로덕션에서는 반드시 치명적 오류로 간주합니다.
    // 로깅 후 안전하게 서버를 재시작하는 전략을 취합니다.
    // 예: cleanupResources(); gracefulShutdown();
    // process.exit(1); // 안전하게 종료
});

// 존재하지 않는 함수를 호출하여 uncaughtException 발생
nonExistentFunction();

ReferenceError: nonExistentFunction is not defined

### **5\. Custom Error 클래스 설계**  

**✔️ 예시 : Custom Error 클래스 설계**

In [4]:
/**
 * 💡 Custom Error 클래스는 반드시 내장 Error 클래스를 상속받아야 합니다.
 */
class AuthenticationError extends Error {
    constructor(message, errorCode = 401) {
        // (1) 부모(Error)의 생성자를 호출하여 name과 message를 설정하고, 스택 트레이스를 기록합니다.
        super(message);

        // (2) 에러 이름을 고정하여 디버깅 시 식별을 용이하게 합니다.
        this.name = "AuthenticationError";

        // (3) Custom 속성(예: HTTP 상태 코드)을 추가하여 정보를 풍부하게 만듭니다.
        this.errorCode = errorCode;
    }
}

class DatabaseConnectionError extends Error {
    constructor(message) {
        super(message);
        this.name = "DatabaseConnectionError";
    }
}

function checkUser(role) {
    if (role !== 'admin') {
        // 특정 Custom Error를 발생시킵니다.
        throw new AuthenticationError(`접근 권한이 없습니다. (현재 역할: ${role})`);
    }
    // ... 정상 로직
}

try {
    checkUser('guest');
} catch (error) {
    // (4) instanceof를 사용하여 에러의 타입을 정확히 식별하고 분기 처리합니다.
    if (error instanceof AuthenticationError) {
        console.warn(`[인증 에러 감지] 사용자에게 로그인 페이지로 리다이렉트 안내 (${error.errorCode})`);
    } else if (error instanceof DatabaseConnectionError) {
        console.error(`[시스템 에러 감지] 데이터베이스 문제 발생`);
    } else {
        // 예상치 못한 다른 모든 에러
        console.error(`[일반 에러] ${error.name}: ${error.message}`);
    }
}

[인증 에러 감지] 사용자에게 로그인 페이지로 리다이렉트 안내 (401)
